# PRUEBAS CON ARF PARA ENTORNO DISTRIBUIDO

In [1]:
from river import forest
from utils import read_dataset, evaluate_model_online_learning


### Read dataset, i.e., have available the dataframe that corresponds to the dataset available 

In [12]:
# Dictionary of dataset names and file paths
data_name = {
    "elec": "electricity.csv",
}

# Load dataset
name = "elec"
dataset = read_dataset(name, data_name)
dataset = dataset.iloc[:5000]


/tmp/ipykernel_11160/228609128.py:24: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dataset.replace({'UP': 1, 'DOWN': 0, 'True': 1, 'False': 0}, inplace=True)


## ARF PARAMETERS
Tras las pruebas se ha visto que el que tiene un mejor equilibrio entre tiempo de ejecucion y rendimiento es con los parametros:
- n_models = 3
- max_size = 50

#### Funciones auxiliares del modelo, extracción de parámetros, agregacion de parámetros etc.

In [13]:
def subtract_parameters(tree):
    """
    Extracts the parameters of a single Hoeffding Tree for sharing with neighbors.

    Args:
        tree: The Hoeffding Tree Classifier.

    Returns:
        A dictionary representing the tree's parameters (nodes, splits, and statistics).
    """
    def traverse(node):
        if node is None:
            return None
        
        # Extract information for the current node
        node_data = {
            "is_leaf": node.is_leaf(),
            "split_feature": getattr(node, "split_feature", None),
            "split_threshold": getattr(node, "split_threshold", None),
            "class_counts": getattr(node, "class_counts", None),
            "left_child": None,
            "right_child": None,
        }
        
        # Recursively extract child nodes
        if not node.is_leaf():
            node_data["left_child"] = traverse(node.left_child)
            node_data["right_child"] = traverse(node.right_child)
        
        return node_data

    # Start traversal from the root
    tree_params = traverse(tree.root)
    return {"nodes": tree_params}


def aggregate_parameters(tree, received_params):
    """
    Aggregates the local Hoeffding Tree parameters with received parameters.

    Args:
        tree: The local Hoeffding Tree Classifier.
        received_params: A list of parameters received from neighbors.

    Returns:
        The updated Hoeffding Tree Classifier.
    """
    def merge_nodes(local_node, neighbor_nodes):
        if local_node is None or all(n is None for n in neighbor_nodes):
            return local_node

        # Merge class counts (leaf nodes)
        if local_node["is_leaf"]:
            for neighbor_node in neighbor_nodes:
                if neighbor_node and neighbor_node["is_leaf"]:
                    for class_label, count in neighbor_node["class_counts"].items():
                        local_node["class_counts"][class_label] = (
                            local_node["class_counts"].get(class_label, 0) + count
                        ) / (len(neighbor_nodes) + 1)

        # Merge split statistics (non-leaf nodes)
        if not local_node["is_leaf"]:
            local_node["left_child"] = merge_nodes(
                local_node["left_child"],
                [n["left_child"] for n in neighbor_nodes if n and not n["is_leaf"]],
            )
            local_node["right_child"] = merge_nodes(
                local_node["right_child"],
                [n["right_child"] for n in neighbor_nodes if n and not n["is_leaf"]],
            )

        return local_node

    # Aggregate received parameters into the local tree
    local_tree_params = subtract_parameters(tree)["nodes"]
    neighbor_tree_params = [params["nodes"] for params in received_params]

    # Merge the root node and update the local tree
    merged_root = merge_nodes(local_tree_params, neighbor_tree_params)
    tree.root = merged_root

    return tree



# CONTINUAR AQUÍ, REWORKED FUNCIONES DE AGREGACION Y EXTRACCION DE PARÁMETROS


#### Vamos a entrenar dos modelos con muestras intercaladas ya que es un dataset con concept drift

In [14]:
# Split the dataset into interleaved chunks
chunk1 = dataset.iloc[::2].reset_index(drop=True)  # Even-indexed samples (0, 2, 4, ...)
chunk2 = dataset.iloc[1::2].reset_index(drop=True)  # Odd-indexed samples (1, 3, 5, ...)

# Initialize the ARF models
modelo1 = forest.ARFClassifier(n_models=3, max_size=20)
modelo2 = forest.ARFClassifier(n_models=3, max_size=20)

# Train modelo1 on chunk1 and modelo2 on chunk2
conf_matrix1, elapsed_time1 = evaluate_model_online_learning(modelo1, chunk1)
conf_matrix2, elapsed_time2 = evaluate_model_online_learning(modelo2, chunk2)

# Print initial evaluation results
print("Initial Evaluation Results:")
print(f"Modelo1 Confusion Matrix: {conf_matrix1}")
print(f"Modelo1 Execution Time: {elapsed_time1:.2f} seconds")
print(f"Modelo2 Confusion Matrix: {conf_matrix2}")
print(f"Modelo2 Execution Time: {elapsed_time2:.2f} seconds")


Initial Evaluation Results:
Modelo1 Confusion Matrix: {'TP': 792, 'TN': 1320, 'FP': 166, 'FN': 222}
Modelo1 Execution Time: 0.92 seconds
Modelo2 Confusion Matrix: {'TP': 688, 'TN': 1390, 'FP': 176, 'FN': 246}
Modelo2 Execution Time: 0.92 seconds


Ahora vamos a printear los parámetros extraídos y antes y después de agregar para comprobar que la agregación se está realizando correctamente y que ambos tienen los mismos parámetros (promedio de los que tenian antes)

In [15]:
params1 = subtract_parameters(modelo1)
params2 = subtract_parameters(modelo2)

print(f"Pametros modelo1:\n {params1}\n\n\n")
print(f"Pametros modelo1:\n {params2}\n\n\n")
 
model1_aggregated = aggregate_parameters(modelo1, params2)
model2_aggregated = aggregate_parameters(modelo2, params1)

params_aggregated1 = subtract_parameters(model1_aggregated)
params_aggregated2 = subtract_parameters(model2_aggregated)

print(f"Pametros modelo1 agregados:\n {params_aggregated1}\n\n\n")
print(f"Pametros modelo2 agregados:\n {params_aggregated2}\n\n\n")

AttributeError: 'ForestMemberClassifier' object has no attribute 'iter_nodes'